Вы работаете в интернет-магазине. Ваша задача проанализировать юнит-экономику и помочь маркетологам разобраться — снижать или увеличивать расходы на маркетинг. У вас есть данные о продажах и расходах. Вы также знаете, что маржинальность магазина равна 40%.

In [8]:
import pandas as pd
import numpy as np

In [9]:
orders = pd.read_csv('ltv_orders_2.csv')
costs = pd.read_csv('ltv_costs_2.csv')

orders['order_date'] = pd.to_datetime(orders['order_date'])
costs['date'] = pd.to_datetime(costs['date'])

orders['order_month'] = orders['order_date'].astype('datetime64[M]')
costs['month'] = costs['date'].astype('datetime64[M]')


first_orders = orders.groupby('uid').agg({'order_month': 'min'}).reset_index()
first_orders.columns = ['uid', 'first_order_month']

cohort_sizes = (
    first_orders.groupby('first_order_month')
    .agg({'uid': 'nunique'})
    .reset_index()
)
cohort_sizes.columns = ['first_order_month', 'n_buyers']
cohort_sizes.head()

margin_rate = 0.4

orders_first_month = pd.merge(orders, first_orders, on='uid')
cohorts = (
    orders_first_month.groupby(['first_order_month', 'order_month'])
    .agg({'revenue': 'sum'})
    .reset_index()
)
report = pd.merge(cohort_sizes, cohorts, on='first_order_month')

report['gp'] = report['revenue'] * margin_rate
report['age'] = (
    report['order_month'] - report['first_order_month']
) / np.timedelta64(1, 'M')
report['age'] = report['age'].round().astype('int')

report['ltv'] = report['gp'] / report['n_buyers']

result = report.pivot_table(
    index='first_order_month', columns='age', values='ltv', aggfunc='mean'
).round()

monthly_costs = costs.groupby('month').sum()
report_new = pd.merge(
    report, monthly_costs, left_on='first_order_month', right_on='month'
)
report_new['cac'] = report_new['costs'] / report_new['n_buyers']

report_new['romi'] = report_new['ltv'] / report_new['cac']
output = report_new.pivot_table(
    index='first_order_month', columns='age', values='romi', aggfunc='mean'
)

final_result=report_new.pivot_table(
    index='first_order_month',
    columns='age',
    values='ltv',
    aggfunc='mean')

m6_cum_ltv = final_result.cumsum(axis=1).mean(axis=0)[5]

print('Средний LTV за 6 месяцев после первой покупки:', m6_cum_ltv)

Средний LTV за 6 месяцев после первой покупки: 2358.917911699909


**Вывод**
Чтобы вложения в маркетинг окупались, привлечение одного покупателя не должно превышать  2358.9 рублей. Полезная информация, маркетологи вам очень благодарны!